In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from sklearn.metrics import f1_score, recall_score, precision_score

from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method as fgsm

In [ ]:
df_train_1 = pd.read_csv("https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/df_train_0.csv", header=0, index_col=0)
df_train_2 = pd.read_csv("https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/df_train_1.csv", header=0, index_col=0)

df_train = pd.concat([df_train_1, df_train_2], axis=0, ignore_index=True)

df_test = pd.read_csv("https://raw.githubusercontent.com/Tdjaaleb/IDS_AdversarialML/main/Data/df_test.csv", header=0, index_col=0)

# Multilayer Perceptron

In [ ]:
X_train = df_train.iloc[:,0:93]
y_train = df_train[['Dos','normal','Probe','R2L','U2R']]

X_test = df_test.iloc[:,0:93]
y_test = df_test[['Dos','normal','Probe','R2L','U2R']]

In [ ]:
#MLP
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Dense(units=50, activation='relu'))
mlp.add(tf.keras.layers.Dense(units=5, activation='softmax'))
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = mlp.fit(X_train, y_train, epochs=100, batch_size=5000,validation_split=0.2)

mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

test_results = mlp.evaluate(X_test, y_test, verbose=1)

pred = mlp.predict(X_test)
for j in range(0,pred.shape[1]):
  for i in range(0,pred.shape[0]):
    pred[i][j] = int(round(pred[i][j]))
pred_df = pd.DataFrame(pred,columns=y_test.columns)

print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
print("Recall Score - ",recall_score(y_test,pred_df.astype('uint8'),average='micro'))
print("F1 Score - ",f1_score(y_test,pred_df.astype('uint8'),average='micro'))
print("Precision Score - ",precision_score(y_test,pred_df.astype('uint8'),average='micro'))

# FGSM

In [ ]:
X_train_pois = fgsm(
    model_fn = mlp,
    x = X_train,
    eps = 0.2,
    norm = np.inf
)

In [ ]:
mlp = tf.keras.Sequential()
mlp.add(tf.keras.layers.Dense(units=50, activation='relu'))
mlp.add(tf.keras.layers.Dense(units=5, activation='softmax'))
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.fit(X_train_pois, y_train, epochs=100, batch_size=5000,validation_split=0.2)

In [ ]:
mlp.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

test_results = mlp.evaluate(X_test, y_test, verbose=1)

pred = mlp.predict(X_test)
for j in range(0,pred.shape[1]):
  for i in range(0,pred.shape[0]):
    pred[i][j] = int(round(pred[i][j]))
pred_df = pd.DataFrame(pred,columns=y_test.columns)

print(f'Test results - Loss: {test_results[0]} - Accuracy: {test_results[1]*100}%')
print("Recall Score - ",recall_score(y_test,pred_df.astype('uint8'),average='micro'))
print("F1 Score - ",f1_score(y_test,pred_df.astype('uint8'),average='micro'))
print("Precision Score - ",precision_score(y_test,pred_df.astype('uint8'),average='micro'))